In [ ]:
import pandas as pd
import numpy as np
import pywhatkit as wpp
from datetime import datetime, timedelta

#SetOptions para ver los dataframes
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

In [ ]:
def dias_alertas():
    hoy = datetime.now()
    dias_semana = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}

    # Encuentra el último martes y jueves
    last_tuesday = hoy - timedelta(days=(hoy.weekday() - dias_semana['martes']) % 7)
    last_thursday = hoy - timedelta(days=(hoy.weekday() - dias_semana['jueves']) % 7)

    # Devuelve la fecha más reciente entre el último martes y jueves
    return max(last_tuesday, last_thursday).strftime('%d')

In [ ]:
def clean_alertas(df):
    # Adios a los registros. Filtro solo Cadereyta
    df = df.drop(columns=['Registro 1', 'Registro 2', 'Registro 3', 'Registro 4', 'Registro 5', 'Subproyecto'])
    df = df[df['Centro Enlace'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]
    
    # Juntar las materias
    # inactivos_agrup = df.groupby(["Matricula"])["Grupo"].agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
    inactivos_agrup = df.groupby(["Matricula", 'Estatus'])["Grupo"].transform(lambda x: ', '.join(x)) #.agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
    df = pd.merge(df, inactivos_agrup, on = "Matricula", how = "left")
    df.shape #Materias Agrupadas en la última colunmna
    
    # Reacomodo de las materias
    columna_extra = df["Materias"]
    df.drop(columns = ["Materias", 'Modelo'], inplace=True)
    df["Grupo"] = columna_extra
    df = df.drop_duplicates(subset='Nombre')
    df.shape #sin duplicados 
    return df

In [ ]:
def clean_database(db):
    #Renombrar el nombre de la columna NOMBRE
    db.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE", 'MATRICULA' : "Matricula"}, inplace= True)

    # Columnas importantes
    db = db[['NOMBRE', 'Matricula', 'CE CANALIZADO', 'CARRERA', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

    # Filtro Cadereyta
    # db = db[db['CE CANALIZADO'] == 'CE CADEREYTA']
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    db = db[db['CE CANALIZADO'].str.contains('|'.join(cade), case = False, na = False)]
    return db

In [ ]:
path = 'Data/'

try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")
    
try: 
    alertas = pd.read_csv(path + "Alertascrudo.csv")
except FileNotFoundError or FileExistsError:
    alertas = pd.read_excel(path + 'Main/Alertas tempranas 30 de Abril 2024.xlsx', sheet_name='General')
    # alertas = clean_alertas(alertas)
    alertas.to_csv(path + 'Alertascrudo.csv', index=False)
    print('Alertas csv Created')


In [ ]:
imagen = 'img/Cerro.jpeg'
mencion_stat = ''
path2 = 'infodata/'

#Inicializar la infodata
try:
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2+ 'matriculas_yacontactadas.csv') 
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_error = pd.DataFrame()
    matric_yacontactadas = pd.DataFrame()

In [ ]:

# Adios a los registros. Filtro solo Cadereyta
df = alertas
df = df.drop(columns=['Registro 1', 'Registro 2', 'Registro 3', 'Registro 4', 'Registro 5', 'Subproyecto'])
df = df[df['Centro Enlace'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]

df['Materias'] = df.groupby(['Matricula', 'Estatus'])["Grupo"].transform(lambda x: ', '.join(x)) #.agg

column_extra = df['Grupo']
df['Grupo'] = df['Materias']

df.drop(columns = ["Materias"], inplace=True)
df.rename(columns={'Grupo' : 'Materias'}, inplace= True)
df.drop_duplicates(subset= ['Matricula', 'Estatus'])